In [11]:
import os
import sys 

parent_directory = os.path.dirname(os.path.abspath(os.getcwd()))
sys.path.append(parent_directory)  # this allows for the pipeline to be imported 

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from pipeline import clean_text
from models import Marivate_model, create_Bert

In [13]:
df = pd.read_csv("fake_and_real_news_dataset.csv")
df.head()

,idd,title,text,label
0,Fq+C96tcx+,‘A target on Roe v. Wade ’: Oklahoma bill maki...,UPDATE: Gov. Fallin vetoed the bill on Friday....,REAL
1,bHUqK!pgmv,Study: women had to drive 4 times farther afte...,Ever since Texas laws closed about half of the...,REAL
2,4Y4Ubf%aTi,"Trump, Clinton clash in dueling DC speeches","Donald Trump and Hillary Clinton, now at the s...",REAL
3,_CoY89SJ@K,Grand jury in Texas indicts activists behind P...,A Houston grand jury investigating criminal al...,REAL
4,+rJHoRQVLe,"As Reproductive Rights Hang In The Balance, De...",WASHINGTON -- Forty-three years after the Supr...,REAL


In [14]:
# drop nan
df.dropna(inplace=True, axis=0)
df.head()

,idd,title,text,label
0,Fq+C96tcx+,‘A target on Roe v. Wade ’: Oklahoma bill maki...,UPDATE: Gov. Fallin vetoed the bill on Friday....,REAL
1,bHUqK!pgmv,Study: women had to drive 4 times farther afte...,Ever since Texas laws closed about half of the...,REAL
2,4Y4Ubf%aTi,"Trump, Clinton clash in dueling DC speeches","Donald Trump and Hillary Clinton, now at the s...",REAL
3,_CoY89SJ@K,Grand jury in Texas indicts activists behind P...,A Houston grand jury investigating criminal al...,REAL
4,+rJHoRQVLe,"As Reproductive Rights Hang In The Balance, De...",WASHINGTON -- Forty-three years after the Supr...,REAL


In [15]:
x = df["title"]+ "." +df["text"]

y = df["label"].apply(lambda x: 1 if x == "REAL" else 0)

In [16]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, random_state=42)

Lstm_train = x_test.apply(clean_text)

In [17]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

full_text_length = x.apply(lambda x: len(x.split()))     # split it into indivdual words, then check

max_length = full_text_length.max()


token = Tokenizer(num_words=10000, oov_token="<OOV>")
token.fit_on_texts(Lstm_train)
word_index = token.word_index

# text processing 
training_seq = token.texts_to_sequences(Lstm_train)

#paqdding 
train_padded = pad_sequences(training_seq,  maxlen=10000,  padding='post')

In [18]:
def addPath(folder_name, list):
    parent_directory = os.path.dirname(os.path.abspath(os.getcwd()))
    folder = os.path.join(parent_directory, folder_name)
    return [os.path.join(folder, i) for i in list]  

In [19]:
def train_LSTM(x, y):
    chosen_weights = ["ww_merge.weights.h5", "WordNet_merge.weights.h5", "Word2Vec_merge.weights.h5", "unbalnced_merge.weights.h5"]

    weights = addPath("LSTM models", chosen_weights)
    
    models = [Marivate_model() for i in range(4)]

    for i in models:
        i.build(input_shape=(None, 10000))

    for index, v in enumerate(weights):
        models[index].load_weights(v)


    for i, v in enumerate(["ww", "wordNet", "word2Vec", "unbalanced"]):
        print(f"Training for: {v}")
        total = 0
        for _ in range(5):
            results = models[i].evaluate(train_padded, y_test, verbose=0)
            total += results[1]
        print(f"Results :{total /5}")
        print("-"*20)

In [20]:
import tensorflow as tf

def train_bert(x, y):
    chosen_models = ["bert_model_ww_merge.keras", "bert_model_WordNet_merge.keras", "bert_model_Word2Vec_merge.keras", "bert_model_unbalnced_merge.keras"]
    models = addPath("BERT models", chosen_models)
    
    berts = [tf.keras.models.load_model(i) for i in models]

    for i, v in enumerate(["ww", "wordNet", "word2Vec", "unbalanced"]):
        print(f"Training for: {v}")
        total = 0
        for _ in range(5):
            results = berts[i].evaluate(x_test, y_test, verbose=0)
            total += results[1]
        print(f"Results :{total /5}")
        print("-"*20)

In [21]:
print("Testing LSTM")
train_LSTM(train_padded, y_test)
print("Testing BERT")
train_bert(x_test, y_test)

Testing LSTM


/mnt/c/Users/g21m3012/Desktop/SA_fakenews/ENV/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 44 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Training for: ww
Results :0.46354731917381287
--------------------
Training for: wordNet
Results :0.5761697292327881
--------------------
Training for: word2Vec
Results :0.5103373527526855
--------------------
Training for: unbalanced
Results :0.5391730070114136
--------------------
Testing BERT


/mnt/c/Users/g21m3012/Desktop/SA_fakenews/ENV/lib/python3.10/site-packages/keras/src/saving/serialization_lib.py:734: UserWarning: `compile()` was not called as part of model loading because the model's `compile()` method is custom. All subclassed Models that have `compile()` overridden should also override `get_compile_config()` and `compile_from_config(config)`. Alternatively, you can call `compile()` manually after loading.
  instance.compile_from_config(compile_config)
/mnt/c/Users/g21m3012/Desktop/SA_fakenews/ENV/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 84 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Training for: ww


2024-10-11 15:03:59.570675: E tensorflow/core/util/util.cc:131] oneDNN supports DT_INT64 only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


Results :0.4532099962234497
--------------------
Training for: wordNet
Results :0.4064200222492218
--------------------
Training for: word2Vec
Results :0.5239390730857849
--------------------
Training for: unbalanced
Results :0.44232863187789917
--------------------


In [22]:
# base training
import keras_nlp

base = keras_nlp.models.BertClassifier.from_preset(
    "bert_tiny_en_uncased",
    num_classes=2,
)

accuracy = base.evaluate(x_test, y_test)[0]
print(f"BERT Base: {accuracy}")

58/58 ━━━━━━━━━━━━━━━━━━━━ 8s 128ms/step - loss: 0.6939 - sparse_categorical_accuracy: 0.4917
BERT Base: 0.6944521069526672
